In [1]:
from collections import Counter
import PIL
from PIL import Image
from matplotlib import pyplot as plt
import os
import random
import numpy as np
import pandas as pd

with open("../domain-generalization-for-anomaly-detection/config.yml", 'r', encoding="utf-8") as f:
    import yaml
    config = yaml.load(f.read(), Loader=yaml.FullLoader)
class_to_idx = config["PACS_class_to_idx"]
domain_to_idx = config["PACS_domain_to_idx"]

In [2]:
data_count_list = []
data_path = "/home/hzw/DGAD/domain-generalization-for-anomaly-detection/data/pacs/semi-supervised/3domain"
file_list = os.listdir(data_path)
for file in sorted(file_list):
    data = np.load(f"{data_path}/{file}", allow_pickle=True)
    tmp = None
    for domain in ["photo", "art_painting", "cartoon", "sketch"]:
        if tmp is None:
            tmp = Counter(data[f"test_{domain}_labels"])
        else:
            tmp += Counter(data[f"test_{domain}_labels"])
    print(config["PACS_idx_to_class"][int(file.split("-")[2])], Counter(data["train_labels"]), Counter(data["val_labels"]), tmp)
    data_count_list.append([config["PACS_idx_to_class"][int(file.split("-")[2])], Counter(data["train_labels"]), Counter(data["val_labels"]), tmp])

dog Counter({0: 614, 1: 18}) Counter({0: 154, 1: 18}) Counter({0: 343, 1: 24})
elephant Counter({0: 586, 1: 18}) Counter({0: 146, 1: 18}) Counter({0: 330, 1: 24})
giraffe Counter({0: 521, 1: 18}) Counter({0: 130, 1: 18}) Counter({0: 312, 1: 24})
guitar Counter({0: 323, 1: 18}) Counter({0: 82, 1: 18}) Counter({0: 221, 1: 24})
horse Counter({0: 465, 1: 18}) Counter({0: 116, 1: 18}) Counter({0: 306, 1: 24})
house Counter({0: 553, 1: 18}) Counter({0: 138, 1: 18}) Counter({0: 188, 1: 24})
person Counter({0: 824, 1: 18}) Counter({0: 206, 1: 18}) Counter({0: 288, 1: 24})


In [3]:
data_path = "/home/hzw/DGAD/domain-generalization-for-anomaly-detection/data/mvtec/semi-supervised/4domain"
file_list = os.listdir(data_path)
for file in sorted(file_list):
    data = np.load(f"{data_path}/{file}", allow_pickle=True)
    tmp = None
    for domain in ["origin", "brightness", "defocus_blur", "contrast", "gaussian_noise"]:
        if tmp is None:
            tmp = Counter(data[f"test_{domain}_labels"])
        else:
            tmp += Counter(data[f"test_{domain}_labels"])
    print(file.split("-")[2].replace(".npz", ""), Counter(data["train_labels"]), Counter(data["val_labels"]), tmp)
    data_count_list.append([file.split("-")[2].replace(".npz", ""), Counter(data["train_labels"]), Counter(data["val_labels"]), tmp])

bottle Counter({0: 668, 1: 12}) Counter({0: 168, 1: 4}) Counter({1: 299, 0: 100})
cable Counter({0: 716, 1: 16}) Counter({0: 180, 1: 4}) Counter({1: 440, 0: 290})
capsule Counter({0: 700, 1: 16}) Counter({0: 176, 1: 4}) Counter({1: 525, 0: 115})
carpet Counter({0: 896, 1: 16}) Counter({0: 224, 1: 4}) Counter({1: 425, 0: 140})
grid Counter({0: 844, 1: 16}) Counter({0: 212, 1: 4}) Counter({1: 265, 0: 105})
hazelnut Counter({0: 1251, 1: 24}) Counter({0: 313, 1: 8}) Counter({1: 318, 0: 200})
leather Counter({0: 784, 1: 16}) Counter({0: 196, 1: 4}) Counter({1: 440, 0: 160})
metal_nut Counter({0: 704, 1: 16}) Counter({0: 176, 1: 4}) Counter({1: 445, 0: 110})
pill Counter({0: 854, 1: 16}) Counter({0: 214, 1: 4}) Counter({1: 685, 0: 130})
screw Counter({0: 1024, 1: 20}) Counter({0: 256, 1: 4}) Counter({1: 571, 0: 205})
tile Counter({0: 736, 1: 16}) Counter({0: 184, 1: 4}) Counter({1: 400, 0: 165})
toothbrush Counter({0: 192, 1: 4}) Counter({0: 48, 1: 4}) Counter({1: 142, 0: 60})
transistor Cou

In [4]:
data_path = "/home/hzw/DGAD/domain-generalization-for-anomaly-detection/data/MNIST/semi-supervised/3domain"
file = "20250120-MNIST-MNIST,MNIST_M,SVHN-0-123456789-216.npz"
data = np.load(f"{data_path}/{file}", allow_pickle=True)
tmp = None
for domain in ["MNIST", "MNIST_M", "SVHN", "SYN"]:
    if tmp is None:
        tmp = Counter(data[f"test_{domain}_labels"])
    else:
        tmp += Counter(data[f"test_{domain}_labels"])
print("MNIST", Counter(data["train_labels"]), Counter(data["val_labels"]), tmp)
data_count_list.append(["MNIST", Counter(data["train_labels"]), Counter(data["val_labels"]), tmp])

MNIST Counter({0: 13356, 1: 270}) Counter({0: 3340, 1: 81}) Counter({0: 3802, 1: 90})


In [5]:
for i in range(len(data_count_list)):
    for j in range(1, len(data_count_list[i])):
        data_count_list[i].append(data_count_list[i][j][0])
        data_count_list[i].append(data_count_list[i][j][1])

In [16]:
data_count = pd.DataFrame(data_count_list, dtype = str).drop([1, 2, 3], axis = 1)
data_count.index = data_count.iloc[:, 0].replace("_", "\\_", regex=True)
data_count.drop(0, axis = 1, inplace=True)
data_count.columns = ["train_noraml", "train_anomaly", "val_normal", "val_anomaly", "test_normal", "test_anomaly"]
data_count.index.name = "data_name"
data_count

,train_noraml,train_anomaly,val_normal,val_anomaly,test_normal,test_anomaly
data_name,,,,,,
dog,614,18,154,18,343,24
elephant,586,18,146,18,330,24
giraffe,521,18,130,18,312,24
guitar,323,18,82,18,221,24
horse,465,18,116,18,306,24
house,553,18,138,18,188,24
person,824,18,206,18,288,24
bottle,668,12,168,4,100,299
cable,716,16,180,4,290,440


In [14]:
out = ""

for name, value in data_count.iterrows():
    out += name + " & " + " & ".join(value.values) + "\t\\\\\n\n"
    out += "\\hline\n\n"
out

'dog & 614 & 18 & 154 & 18 & 343 & 24\t\\\\\n\n\\hline\n\nelephant & 586 & 18 & 146 & 18 & 330 & 24\t\\\\\n\n\\hline\n\ngiraffe & 521 & 18 & 130 & 18 & 312 & 24\t\\\\\n\n\\hline\n\nguitar & 323 & 18 & 82 & 18 & 221 & 24\t\\\\\n\n\\hline\n\nhorse & 465 & 18 & 116 & 18 & 306 & 24\t\\\\\n\n\\hline\n\nhouse & 553 & 18 & 138 & 18 & 188 & 24\t\\\\\n\n\\hline\n\nperson & 824 & 18 & 206 & 18 & 288 & 24\t\\\\\n\n\\hline\n\nbottle & 668 & 12 & 168 & 4 & 100 & 299\t\\\\\n\n\\hline\n\ncable & 716 & 16 & 180 & 4 & 290 & 440\t\\\\\n\n\\hline\n\ncapsule & 700 & 16 & 176 & 4 & 115 & 525\t\\\\\n\n\\hline\n\ncarpet & 896 & 16 & 224 & 4 & 140 & 425\t\\\\\n\n\\hline\n\ngrid & 844 & 16 & 212 & 4 & 105 & 265\t\\\\\n\n\\hline\n\nhazelnut & 1251 & 24 & 313 & 8 & 200 & 318\t\\\\\n\n\\hline\n\nleather & 784 & 16 & 196 & 4 & 160 & 440\t\\\\\n\n\\hline\n\nmetal_nut & 704 & 16 & 176 & 4 & 110 & 445\t\\\\\n\n\\hline\n\npill & 854 & 16 & 214 & 4 & 130 & 685\t\\\\\n\n\\hline\n\nscrew & 1024 & 20 & 256 & 4 & 205 & 571

In [11]:
with open("temp.txt","w") as f:
    f.write(out)